# Project: Using NASDAQ data

This project uses data from https://data.nasdaq.com/ (formerly Quandl API). The API key is stored in an .env file. 

In [10]:
# Import the relevant modules
import requests
import collections
import os
import json

from dotenv import load_dotenv  # if missing this module, simply run `pip install python-dotenv`

# get api key from your .env file
load_dotenv()
API_KEY = os.getenv('NASDAQ_API_KEY')

Note: API's can change a bit with each version, for this exercise it is reccomended to use the nasdaq api at `https://data.nasdaq.com/api/v3/`. This is the same api as what used to be quandl so `https://www.quandl.com/api/v3/` should work too.

Hint: We are looking for the `AFX_X` data on the `datasets/FSE/` dataset.

<h3>Example</h3>

In [11]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
dataset_code = "WIKI/AP"
start_date = "2023-12-15"
end_date = "2023-12-15"


# Replace "YOUR_API_KEY" with your actual Nasdaq Data Link API key
url = f"https://data.nasdaq.com/api/v3/datasets/{dataset_code}?start_date={start_date}&end_date={end_date}&api_key={API_KEY}"

response = requests.get(url)
response.raise_for_status()  # Raise an error if the request fails

data = response.json()

<p>Inspect JSON structure</p>

In [12]:
# Inspect the JSON structure of the object you created, and take note of how nested it is,
# as well as the overall structure

print(json.dumps(
    data,
    sort_keys=True,
    indent=4,
    separators=(',', ': ')
))

{
    "dataset": {
        "collapse": null,
        "column_index": null,
        "column_names": [
            "Date",
            "Open",
            "High",
            "Low",
            "Close",
            "Volume",
            "Ex-Dividend",
            "Split Ratio",
            "Adj. Open",
            "Adj. High",
            "Adj. Low",
            "Adj. Close",
            "Adj. Volume"
        ],
        "data": [],
        "database_code": "WIKI",
        "database_id": 4922,
        "dataset_code": "AP",
        "description": "<p>End of day open, high, low, close and volume, dividends and splits, and split/dividend adjusted open, high, low close and volume for Ampco-Pittsburgh Corporation (AP). Ex-Dividend is non-zero on ex-dividend dates. Split Ratio is 1 on non-split dates. Adjusted prices are calculated per CRSP (<a href=\"http://www.crsp.com/products/documentation/crsp-calculations\" rel=\"nofollow\" target=\"blank\">www.crsp.com/products/documentation/crsp-calcula

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

## 1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD)

In [13]:
start_date = "2017-01-01"
end_date = "2017-12-31"
dataset_code = "FSE/AFX_X"
url = f"https://data.nasdaq.com/api/v3/datasets/{dataset_code}?start_date={start_date}&end_date={end_date}&api_key={API_KEY}"

response = requests.get(url)
response.raise_for_status()  # Raise an error if the request fails

## 2. Convert the returned JSON object into a Python dictionary.

In [26]:
data = response.json()
#print(json.dumps(data, sort_keys=True, indent=4, separators=(',', ': ')))
print(type(data))

<class 'dict'>


## 3. Calculate what the highest and lowest opening prices were for the stock in this period.

In [15]:

open_prices = [daily[1] for daily in data['dataset']['data'] if daily[1] != None]
max_open = str(max(open_prices))
min_open = str(min(open_prices))
print('The highest opening price was $' + max_open + ' and the lowest opening price was $' + min_open + ' in 2017.')

The highest opening price was $53.11 and the lowest opening price was $34.0 in 2017.


## 4. What was the largest change in any one day (based on High and Low price)?

In [16]:
#Find the index of high and low prices
high = data['dataset']['column_names'].index('High')
low = data['dataset']['column_names'].index('Low')
#Calculate change
change = [day[high]-day[low] for day in data['dataset']['data']]
print('The largest change in any one day in 2017 was {:.2f}'.format(max(change)))

The largest change in any one day in 2017 was 2.81


## 5. What was the largest change between any two days (based on Closing Price)?

In [18]:
#Find the index of closing price
close = data['dataset']['column_names'].index('Close')
#Grab closing price
closing = [day[close] for day in data['dataset']['data']]
#Calculate change in price between day and the day before loop
change_two_days = [abs(closing[index]-closing[index-1]) for index, value in enumerate(closing) if index > 0]
print('The largest change between any two days in 2017 was {:.2f}'.format(max(change_two_days)))

The largest change between any two days in 2017 was 2.56


## 6. What was the average daily trading volume during this year?

In [19]:
#Find the index of volume
traded_volume = data['dataset']['column_names'].index('Traded Volume')
#Extract trading volume
volume = [day[traded_volume] for day in data['dataset']['data']]
#Calculate average total volume divided by total number
average_volume = sum(volume)/len(volume)
print('The average daily trading volumne in 2017 was {:.2f}'.format(average_volume))

The average daily trading volumne in 2017 was 89124.34


## 7. What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [20]:
#Sort volume ascending
volume = sorted(volume)
#Find median through analog calculations
if len(volume) % 2 == 1:
    median_volume = volume[len(volume)//2]
else:
    median_volume = (volume[len(volume)//2] + volume[len(volume)//2-1]) / 2
print('The median trading volume in 2017 was {:.2f}'.format(median_volume))

The median trading volume in 2017 was 76286.00
